In [3]:
import pandas as pd
pd.options.display.precision = 4

In [3]:
df = pd.read_csv("../outputs/PostHoc/figures/FashionMNIST/fixed/metrics.csv",
                 header=None,
                 names=["metric", "value", "latent_size"],
                 dtype={"metric": str, "value": float, "latent_size": int})

df\
    .sort_values(by=["latent_size", "metric"])\
    .groupby(["latent_size", "metric"]).agg(["mean", "std"])

df = df\
    .sort_values(by=["latent_size", "metric"])\
    .groupby(["latent_size", "metric"]).agg(["mean", "std"])\

df.columns = df.columns.to_flat_index()

df = df.assign(val=lambda x: x[("value", "mean")].round(4).apply(str) + " ± " + x[("value", "std")].round(4).apply(str))\
    .drop([("value", "mean"), ("value", "std")], axis=1)\
    .reset_index()\
    .pivot(index="latent_size", columns="metric", values="val")

cols = ["precision_at_1", "mean_average_precision", "recall_at_k", "auroc", "auprc", "ausc", "ece", "precision_at_1_expected", "mean_average_precision_expected", "recall_at_k_expected"]
df[cols]


metric,precision_at_1,mean_average_precision,recall_at_k,auroc,auprc,ausc,ece,precision_at_1_expected,mean_average_precision_expected,recall_at_k_expected
latent_size,,,,,,,,,,
2,0.7579 ± 0.002,0.8085 ± 0.0016,0.9149 ± 0.0019,0.9888 ± 0.0002,0.9872 ± 0.0003,0.8587 ± 0.003,0.0425 ± 0.0038,0.6793 ± 0.0346,0.7519 ± 0.0208,0.9029 ± 0.0291
16,0.9068 ± 0.0008,0.9192 ± 0.0005,0.947 ± 0.0014,0.9667 ± 0.0002,0.9612 ± 0.0006,0.9618 ± 0.0004,0.0862 ± 0.0019,0.9015 ± 0.0091,0.9144 ± 0.0046,0.9435 ± 0.0025
32,0.9098 ± 0.0013,0.9214 ± 0.0005,0.9468 ± 0.0007,0.9592 ± 0.0007,0.9518 ± 0.0008,0.9647 ± 0.0004,0.1317 ± 0.0043,0.8984 ± 0.0057,0.9121 ± 0.0027,0.9474 ± 0.0048


In [2]:
df = pd.read_csv("../outputs/PostHoc/figures/CIFAR10/full/metrics.csv",
                 header=None,
                 names=["metric", "value", "latent_size"],
                 dtype={"metric": str, "value": float, "latent_size": int})

df\
    .sort_values(by=["latent_size", "metric"])\
    .groupby(["latent_size", "metric"]).agg(["mean", "std"])

df = df\
    .sort_values(by=["latent_size", "metric"])\
    .groupby(["latent_size", "metric"]).agg(["mean", "std"])\

df.columns = df.columns.to_flat_index()

df = df.assign(val=lambda x: x[("value", "mean")].round(4).apply(str) + " ± " + x[("value", "std")].round(4).apply(str))\
    .drop([("value", "mean"), ("value", "std")], axis=1)\
    .reset_index()\
    .pivot(index="latent_size", columns="metric", values="val")

cols = ["precision_at_1", "mean_average_precision", "recall_at_k", "auroc", "auprc", "ausc", "ece", "precision_at_1_expected", "mean_average_precision_expected", "recall_at_k_expected"]
df[cols]


metric,precision_at_1,mean_average_precision,recall_at_k,auroc,auprc,ausc,ece,precision_at_1_expected,mean_average_precision_expected,recall_at_k_expected
latent_size,,,,,,,,,,
16,0.5891 ± 0.0051,0.6543 ± 0.0035,0.7853 ± 0.0031,0.6097 ± 0.0231,0.7508 ± 0.018,0.77 ± 0.005,0.128 ± 0.0059,0.6297 ± 0.0294,0.6534 ± 0.0126,0.7013 ± 0.0107
32,0.5958 ± 0.0045,0.6579 ± 0.0022,0.7838 ± 0.0041,0.6337 ± 0.0129,0.7661 ± 0.0088,0.7709 ± 0.0019,0.2027 ± 0.0095,0.6374 ± 0.0063,0.649 ± 0.0075,0.6824 ± 0.0112
64,0.6178 ± 0.0041,0.6603 ± 0.0036,0.747 ± 0.0039,0.5568 ± 0.0145,0.712 ± 0.009,0.7647 ± 0.0026,0.2152 ± 0.0086,0.522 ± 0.0299,0.55 ± 0.0209,0.6124 ± 0.0222


# Online

In [4]:
from pathlib import Path
import json

dfs = []
for folder in Path("../outputs/Online/figures/FashionMNIST").glob("*"):
    method, latent_size, _, seed = folder.name.split("_")

    df = pd.read_csv(folder / "metrics.csv",
                     header=None,
                     names=["metric", "value", "latent_size"],
                     dtype={"metric": str, "value": float, "latent_size": int})
    if len(df) > 12:
        df = df[-8:]
    print(df)
    ood_metrics = json.load(open(folder / "ood_metrics.json"))
    ood_metrics = pd.DataFrame({"metric": ["auroc", "auprc"], "value": [ood_metrics["auroc"], ood_metrics["auprc"]]})
    df = pd.concat([df, ood_metrics], axis=0)
    df["seed"] = int(seed)
    df["latent_size"] = int(latent_size)
    df["method"] = method
    dfs.append(df)
df = pd.concat(dfs)
df["method"] = df["method"].str.replace("fixed", "A: fixed")
df["method"] = df["method"].str.replace("positives", "B: positives")
df["method"] = df["method"].str.replace("full", "C: full")

df = df\
    .sort_values(by=["latent_size", "metric"])\
    .groupby(["method", "latent_size", "metric"]).agg(["mean", "std"])\

df.columns = df.columns.to_flat_index()

df = df.assign(val=lambda x: x[("value", "mean")].round(4).apply(str) + " ± " + x[("value", "std")].round(4).apply(str))\
    .drop([("value", "mean"), ("value", "std")], axis=1)\
    .reset_index()\
    .pivot(index=["latent_size", "method"], columns="metric", values="val")

cols = ["precision_at_1", "mean_average_precision", "recall_at_k", "auroc", "auprc", "ausc", "ece", "precision_at_1_expected", "mean_average_precision_expected", "recall_at_k_expected"]
df[cols]

                             metric   value  latent_size
10           mean_average_precision  0.7803            2
11                   precision_at_1  0.7358            2
12                      recall_at_k  0.9003            2
13  mean_average_precision_expected  0.6283            2
14          precision_at_1_expected  0.3900            2
15             recall_at_k_expected  0.9380            2
16                              ece  0.0578            2
17                             ausc  0.6208            2
                             metric   value  latent_size
10           mean_average_precision  0.8778           32
11                   precision_at_1  0.8510           32
12                      recall_at_k  0.9352           32
13  mean_average_precision_expected  0.8163           32
14          precision_at_1_expected  0.7452           32
15             recall_at_k_expected  0.9203           32
16                              ece  0.2754           32
17                             

metric                     precision_at_1 mean_average_precision  \
latent_size method                                                 
2           A: fixed      0.7343 ± 0.0182        0.7933 ± 0.0104   
            B: positives  0.7526 ± 0.0128         0.8038 ± 0.009   
            C: full       0.7156 ± 0.0322        0.7777 ± 0.0182   
16          A: fixed      0.8145 ± 0.0182        0.8534 ± 0.0088   
            B: positives  0.8238 ± 0.0151        0.8578 ± 0.0114   
            C: full       0.8044 ± 0.0212        0.8508 ± 0.0065   
32          A: fixed      0.8279 ± 0.0149        0.8624 ± 0.0104   
            B: positives  0.8222 ± 0.0328        0.8632 ± 0.0123   
            C: full       0.8356 ± 0.0085         0.865 ± 0.0043   

metric                        recall_at_k            auroc            auprc  \
latent_size method                                                            
2           A: fixed      0.9187 ± 0.0053  0.9857 ± 0.0067  0.9837 ± 0.0065   
            B: positives  0.9148 ± 0.0035  0.9832 ± 0.0044  0.9789 ± 0.0045   
            C: full       0.9148 ± 0.0103  0.9358 ± 0.0731  0.9442 ± 0.0557   
16          A: fixed      0.9329 ± 0.0029  0.9385 ± 0.0649  0.9506 ± 0.0507   
            B: positives   0.935 ± 0.0032  0.7813 ± 0.2123  0.8044 ± 0.2025   
            C: full       0.9345 ± 0.0027  0.8689 ± 0.1306  0.8918 ± 0.0974   
32          A: fixed      0.9349 ± 0.0034  0.9351 ± 0.0343  0.9452 ± 0.0243   
            B: positives   0.937 ± 0.0031   0.8456 ± 0.165  0.8538 ± 0.1674   
            C: full       0.9381 ± 0.0018  0.8856 ± 0.0501  0.9009 ± 0.0397   

metric                               ausc              ece  \
latent_size method                                           
2           A: fixed      0.8639 ± 0.0252  0.0515 ± 0.0159   
            B: positives   0.8771 ± 0.009  0.0201 ± 0.0058   
            C: full       0.8213 ± 0.1125  0.0312 ± 0.0179   
16          A: fixed      0.9239 ± 0.0038  0.2523 ± 0.0296   
            B: positives   0.9205 ± 0.022  0.2794 ± 0.0248   
            C: full       0.9167 ± 0.0146  0.2429 ± 0.0239   
32          A: fixed       0.923 ± 0.0144  0.2458 ± 0.0292   
            B: positives   0.928 ± 0.0127   0.266 ± 0.0277   
            C: full       0.9322 ± 0.0085   0.2578 ± 0.016   

metric                   precision_at_1_expected  \
latent_size method                                 
2           A: fixed             0.5459 ± 0.0598   
            B: positives         0.6597 ± 0.0686   
            C: full               0.446 ± 0.1496   
16          A: fixed             0.8084 ± 0.0263   
            B: positives         0.8073 ± 0.0205   
            C: full              0.7724 ± 0.0468   
32          A: fixed             0.7447 ± 0.0459   
            B: positives          0.7668 ± 0.044   
            C: full                0.8054 ± 0.02   

metric                   mean_average_precision_expected recall_at_k_expected  
latent_size method                                                             
2           A: fixed                     0.7039 ± 0.0264      0.9355 ± 0.0197  
            B: positives                 0.7619 ± 0.0394      0.9166 ± 0.0144  
            C: full                      0.6444 ± 0.0724      0.9229 ± 0.0324  
16          A: fixed                     0.8444 ± 0.0091      0.9197 ± 0.0034  
            B: positives                 0.8345 ± 0.0206      0.9211 ± 0.0052  
            C: full                      0.8276 ± 0.0213      0.9224 ± 0.0074  
32          A: fixed                      0.8029 ± 0.031       0.919 ± 0.0096  
            B: positives                  0.809 ± 0.0099      0.9264 ± 0.0054  
            C: full                      0.8274 ± 0.0204      0.9194 ± 0.0093

In [2]:
from pathlib import Path

dfs = []
for folder in Path("../outputs/Online/figures/CIFAR10").glob("*"):
    try:
        method, latent_size, _, seed = folder.name.split("_")

        df = pd.read_csv(folder / "metrics.csv",
                        header=None,
                        names=["metric", "value", "latent_size"],
                        dtype={"metric": str, "value": float, "latent_size": int})
        df["seed"] = int(seed)
        df["latent_size"] = int(latent_size)
        df["method"] = method
        dfs.append(df)
    except:
        continue

df = pd.concat(dfs)
df["method"] = df["method"].str.replace("fixed", "A: fixed")
df["method"] = df["method"].str.replace("positives", "B: positives")
df["method"] = df["method"].str.replace("full", "C: full")

df = df\
    .sort_values(by=["latent_size", "metric"])\
    .groupby(["method", "latent_size", "metric"]).agg(["mean", "std"])\

df.columns = df.columns.to_flat_index()

df = df.assign(val=lambda x: x[("value", "mean")].round(4).apply(str) + " ± " + x[("value", "std")].round(4).apply(str))\
    .drop([("value", "mean"), ("value", "std")], axis=1)\
    .reset_index()\
    .pivot(index=["latent_size", "method"], columns="metric", values="val")

cols = ["precision_at_1", "mean_average_precision", "recall_at_k", "auroc", "auprc", "ausc", "ece", "precision_at_1_expected", "mean_average_precision_expected", "recall_at_k_expected"]
df[cols]

metric                     precision_at_1 mean_average_precision  \
latent_size method                                                 
16          A: fixed      0.5695 ± 0.0064        0.6387 ± 0.0052   
            B: positives  0.5661 ± 0.0076        0.6371 ± 0.0073   
            C: full       0.5579 ± 0.0044         0.6304 ± 0.004   
32          A: fixed      0.5713 ± 0.0035         0.6401 ± 0.004   
            B: positives  0.5711 ± 0.0099        0.6429 ± 0.0061   
            C: full        0.572 ± 0.0069        0.6417 ± 0.0055   
64          A: fixed      0.5734 ± 0.0043        0.6407 ± 0.0027   
            B: positives  0.5665 ± 0.0072        0.6391 ± 0.0041   
            C: full       0.5694 ± 0.0049         0.639 ± 0.0029   

metric                        recall_at_k            auroc            auprc  \
latent_size method                                                            
16          A: fixed       0.786 ± 0.0034  0.4811 ± 0.0209  0.6688 ± 0.0117   
            B: positives   0.7882 ± 0.005  0.4604 ± 0.0328  0.6586 ± 0.0169   
            C: full       0.7825 ± 0.0036  0.4434 ± 0.0255   0.6477 ± 0.012   
32          A: fixed      0.7888 ± 0.0027   0.457 ± 0.0314  0.6525 ± 0.0154   
            B: positives   0.7913 ± 0.005    0.451 ± 0.033  0.6462 ± 0.0176   
            C: full       0.7901 ± 0.0064  0.4251 ± 0.0225  0.6336 ± 0.0107   
64          A: fixed      0.7886 ± 0.0041  0.4554 ± 0.0292   0.646 ± 0.0138   
            B: positives  0.7889 ± 0.0048    0.45 ± 0.0226  0.6431 ± 0.0087   
            C: full       0.7869 ± 0.0038  0.4392 ± 0.0135  0.6377 ± 0.0064   

metric                               ausc              ece  \
latent_size method                                           
16          A: fixed      0.7207 ± 0.0089  0.0347 ± 0.0024   
            B: positives   0.7167 ± 0.013  0.0312 ± 0.0036   
            C: full       0.7073 ± 0.0093   0.0329 ± 0.001   
32          A: fixed      0.7248 ± 0.0063  0.0595 ± 0.0025   
            B: positives  0.7244 ± 0.0056  0.0556 ± 0.0042   
            C: full       0.7161 ± 0.0108  0.0568 ± 0.0062   
64          A: fixed       0.7154 ± 0.006   0.075 ± 0.0084   
            B: positives  0.7172 ± 0.0086  0.0791 ± 0.0089   
            C: full       0.7136 ± 0.0059  0.0756 ± 0.0079   

metric                   precision_at_1_expected  \
latent_size method                                 
16          A: fixed             0.5304 ± 0.0112   
            B: positives          0.5238 ± 0.022   
            C: full               0.508 ± 0.0069   
32          A: fixed             0.4426 ± 0.0226   
            B: positives         0.4454 ± 0.0677   
            C: full              0.4307 ± 0.0505   
64          A: fixed             0.4186 ± 0.0583   
            B: positives         0.4006 ± 0.0382   
            C: full              0.3663 ± 0.0272   

metric                   mean_average_precision_expected recall_at_k_expected  
latent_size method                                                             
16          A: fixed                     0.5544 ± 0.0088       0.623 ± 0.0293  
            B: positives                 0.5481 ± 0.0166      0.6036 ± 0.0169  
            C: full                       0.538 ± 0.0219      0.6102 ± 0.0393  
32          A: fixed                     0.4925 ± 0.0138      0.6084 ± 0.0326  
            B: positives                  0.491 ± 0.0264      0.5876 ± 0.0453  
            C: full                       0.4724 ± 0.035      0.5534 ± 0.0552  
64          A: fixed                     0.4584 ± 0.0522      0.5232 ± 0.0521  
            B: positives                 0.4253 ± 0.0395      0.5124 ± 0.0648  
            C: full                       0.425 ± 0.0534       0.508 ± 0.0956